## Bike Sharing Data Experimnt Tracking

In [95]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
import numpy as np


import warnings
warnings.filterwarnings('ignore')

In [96]:
train = pd.read_csv('train.csv')
bike=train.copy()

In [97]:
train.shape

(10886, 12)

In [98]:
train.describe()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
count,10886.000000,10886.000000,10886.000000,10886.000000,10886.00000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000
mean,2.506614,0.028569,0.680875,1.418427,20.23086,23.655084,61.886460,12.799395,36.021955,155.552177,191.574132
std,1.116174,0.166599,0.466159,0.633839,7.79159,8.474601,19.245033,8.164537,49.960477,151.039033,181.144454
min,1.000000,0.000000,0.000000,1.000000,0.82000,0.760000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,0.000000,1.000000,13.94000,16.665000,47.000000,7.001500,4.000000,36.000000,42.000000
50%,3.000000,0.000000,1.000000,1.000000,20.50000,24.240000,62.000000,12.998000,17.000000,118.000000,145.000000
75%,4.000000,0.000000,1.000000,2.000000,26.24000,31.060000,77.000000,16.997900,49.000000,222.000000,284.000000
max,4.000000,1.000000,1.000000,4.000000,41.00000,45.455000,100.000000,56.996900,367.000000,886.000000,977.000000


In [99]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


In [100]:
train.isnull().sum()

datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
casual        0
registered    0
count         0
dtype: int64

In [101]:
train.head(2)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40


In [102]:
bike['casual'] = bike['casual'].apply(lambda x: x*15)
bike['registered'] = bike['registered'].apply(lambda x: x*10)

bike['total'] = bike['casual'] + bike['registered']

In [103]:
bike.head(2)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,total
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,45,130,16,175
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,120,320,40,440


In [104]:
bike['datetime'] = pd.to_datetime(bike['datetime'])

bike['day'] = bike['datetime'].map(lambda x:x.day)
bike['month'] = bike['datetime'].map(lambda x:x.month)
bike['year'] = bike['datetime'].map(lambda x:x.year)
bike['hour'] = bike['datetime'].map(lambda x:x.hour)

bike.drop(labels=['datetime'], axis=1, inplace=True)

In [105]:
bike.head(2)

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,total,day,month,year,hour
0,1,0,0,1,9.84,14.395,81,0.0,45,130,16,175,1,1,2011,0
1,1,0,0,1,9.02,13.635,80,0.0,120,320,40,440,1,1,2011,1


In [106]:
bike.drop(['count','casual','registered', 'year', 'hour', 'day', 'workingday', 'holiday'],axis=1,inplace=True)

In [107]:
bike.head(2)

,season,weather,temp,atemp,humidity,windspeed,total,month
0,1,1,9.84,14.395,81,0.0,175,1
1,1,1,9.02,13.635,80,0.0,440,1


## OneHotEncoder

In [14]:
bike.season.replace({1:"spring", 2:"summer", 3:"fall", 4:"winter"},inplace = True)

bike.weather.replace({1:'good',2:'moderate',3:'bad',4:'severe'},inplace = True)

bike.month.replace({1: 'jan',2: 'feb',3: 'mar',4: 'apr',5: 'may',6: 'jun',
                  7: 'jul',8: 'aug',9: 'sept',10: 'oct',11: 'nov',12: 'dec'},inplace = True)

In [15]:
bike.head(3)

,season,weather,temp,atemp,humidity,windspeed,total,month
0,spring,good,9.84,14.395,81,0.0,175,jan
1,spring,good,9.02,13.635,80,0.0,440,jan
2,spring,good,9.02,13.635,80,0.0,345,jan


In [16]:
bike = pd.get_dummies(data=bike,columns=["season", 'month'],drop_first=True)
bike = pd.get_dummies(data=bike,columns=["weather"])

## Normalization

In [108]:
min_max=MinMaxScaler()

scaled=pd.DataFrame(min_max.fit_transform(bike), columns=bike.columns)

scaled.head()

,season,weather,temp,atemp,humidity,windspeed,total,month
0,0.0,0.0,0.224490,0.305068,0.81,0.0,0.015670,0.0
1,0.0,0.0,0.204082,0.288064,0.80,0.0,0.040836,0.0
2,0.0,0.0,0.204082,0.288064,0.80,0.0,0.031814,0.0
3,0.0,0.0,0.224490,0.305068,0.75,0.0,0.012821,0.0
4,0.0,0.0,0.224490,0.305068,0.75,0.0,0.000000,0.0


In [140]:
X = scaled.drop(labels=['total'], axis=1)
y = scaled['total']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [141]:
X_train.head(5)

,season,weather,temp,atemp,humidity,windspeed,month
613,0.000000,0.000000,0.204082,0.186374,0.32,0.684260,0.090909
4030,0.666667,0.000000,0.530612,0.559347,0.68,0.228047,0.727273
3582,0.666667,0.000000,0.632653,0.627140,0.83,0.000000,0.636364
10101,1.000000,0.000000,0.204082,0.237275,0.69,0.157870,0.909091
1430,0.333333,0.666667,0.306122,0.305068,0.81,0.526390,0.272727


## Simple Experimnet 

In [142]:
rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)

y_pred = rfr.predict(X_test)

mean_squared_error(y_test, y_pred, squared=False)

0.1535432358241506

## Cross Validation 

In [21]:
cv = RepeatedKFold(n_splits=10, n_repeats=5, random_state=64)
scores = cross_val_score(RandomForestRegressor(), X, y, cv=cv, n_jobs=-1)

print('Accuracy:%.3f ' % (np.mean(scores)))

Accuracy:0.360 



## Mlflow tracking

In [22]:
import mlflow

In [23]:
mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('bike-sharing-experiment')

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='bike-sharing-experiment', tags={}>

In [24]:
with mlflow.start_run():
    mlflow.set_tag("workspace", "project")
    
    max_depth = 5
    mlflow.log_param("max_depth", max_depth)
    
    cv_scores = cross_val_score(RandomForestRegressor(max_depth=max_depth, random_state=65), X_train, y_train, cv=10)

    Accuracy = cv_scores.mean()
    
    mlflow.log_metric("Accuracy", cv_scores.mean())

## Hyperparameters Optimization

In [25]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [26]:
train = xgb.DMatrix(X_train, label=y_train)
test = xgb.DMatrix(X_test, label=y_test)

In [27]:
def objective(params):
    with mlflow.start_run():
        num_boost_round = 500
        early_stopping_rounds = 150
        
        mlflow.log_params(params)
        mlflow.log_param('num_boost_round', num_boost_round)
        mlflow.log_param('early_stopping_rounds', early_stopping_rounds)
        mlflow.set_tag('model', 'xgboost')

        booster = xgb.train(
            params = params,
            dtrain = train,
            evals = [(test, "test")],
            num_boost_round = num_boost_round,
            early_stopping_rounds = early_stopping_rounds
        )
        
        y_pred = booster.predict(test)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        mlflow.log_metric('rmse', rmse)
        return {'loss': rmse, 'status': STATUS_OK}

grid_search = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child': hp.loguniform('min_child', -1, 3),
    'seed': 111,
    'objective': 'reg:linear'
}

In [29]:
best_model = fmin(
    fn=objective,
    space=grid_search,
    algo=tpe.suggest,
    max_evals=30,
    trials=Trials()
)

[11:21:56] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[11:21:56] WARNING: ../src/learner.cc:627:                                      
Parameters: { "min_child" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:0.27160                                                           
[1]	test-rmse:0.22016                                                           
[2]	test-rmse:0.18903                                                           
[3]	test-rmse:0.17208                                                           
[4]	test-rmse:0.16333                                                           
[5]	test-rmse:0.15908                                                          

[94]	test-rmse:0.16455                                                          
[95]	test-rmse:0.16455                                                          
[96]	test-rmse:0.16455                                                          
[97]	test-rmse:0.16455                                                          
[98]	test-rmse:0.16455                                                          
[99]	test-rmse:0.16455                                                          
[100]	test-rmse:0.16455                                                         
[101]	test-rmse:0.16455                                                         
[102]	test-rmse:0.16455                                                         
[103]	test-rmse:0.16455                                                         
[104]	test-rmse:0.16455                                                         
[105]	test-rmse:0.16455                                                         
[106]	test-rmse:0.16455     

[30]	test-rmse:0.16447                                                          
[31]	test-rmse:0.16452                                                          
[32]	test-rmse:0.16460                                                          
[33]	test-rmse:0.16471                                                          
[34]	test-rmse:0.16481                                                          
[35]	test-rmse:0.16488                                                          
[36]	test-rmse:0.16492                                                          
[37]	test-rmse:0.16497                                                          
[38]	test-rmse:0.16502                                                          
[39]	test-rmse:0.16507                                                          
[40]	test-rmse:0.16512                                                          
[41]	test-rmse:0.16519                                                          
[42]	test-rmse:0.16523      

[131]	test-rmse:0.16617                                                         
[132]	test-rmse:0.16617                                                         
[133]	test-rmse:0.16617                                                         
[134]	test-rmse:0.16617                                                         
[135]	test-rmse:0.16617                                                         
[136]	test-rmse:0.16617                                                         
[137]	test-rmse:0.16617                                                         
[138]	test-rmse:0.16617                                                         
[139]	test-rmse:0.16617                                                         
[140]	test-rmse:0.16617                                                         
[141]	test-rmse:0.16617                                                         
[142]	test-rmse:0.16617                                                         
[143]	test-rmse:0.16617     

[67]	test-rmse:0.17161                                                          
[68]	test-rmse:0.17171                                                          
[69]	test-rmse:0.17175                                                          
[70]	test-rmse:0.17186                                                          
[71]	test-rmse:0.17190                                                          
[72]	test-rmse:0.17204                                                          
[73]	test-rmse:0.17211                                                          
[74]	test-rmse:0.17218                                                          
[75]	test-rmse:0.17225                                                          
[76]	test-rmse:0.17235                                                          
[77]	test-rmse:0.17238                                                          
[78]	test-rmse:0.17244                                                          
[79]	test-rmse:0.17250      

...

[54]	test-rmse:0.16725                                                          
[55]	test-rmse:0.16726                                                          
[56]	test-rmse:0.16727                                                          
[57]	test-rmse:0.16728                                                          
[58]	test-rmse:0.16730                                                          
[59]	test-rmse:0.16733                                                          
[60]	test-rmse:0.16734                                                          
[61]	test-rmse:0.16737                                                          
[62]	test-rmse:0.16739                                                          
[63]	test-rmse:0.16741                                                          
[64]	test-rmse:0.16743                                                          
[65]	test-rmse:0.16744                                                          
[66]	test-rmse:0.16745      

[155]	test-rmse:0.16770                                                         
[156]	test-rmse:0.16770                                                         
[11:24:06] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[11:24:06] WARNING: ../src/learner.cc:627:                                      
Parameters: { "min_child" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:0.27203                                                           
[1]	test-rmse:0.21929                                                           
[2]	test-rmse:0.18766                                                           
[3]	test-rmse:0.16982                                                          

[92]	test-rmse:0.14910                                                          
[93]	test-rmse:0.14916                                                          
[94]	test-rmse:0.14906                                                          
[95]	test-rmse:0.14898                                                          
[96]	test-rmse:0.14894                                                          
[97]	test-rmse:0.14896                                                          
[98]	test-rmse:0.14907                                                          
[99]	test-rmse:0.14911                                                          
[100]	test-rmse:0.14914                                                         
[101]	test-rmse:0.14917                                                         
[102]	test-rmse:0.14916                                                         
[103]	test-rmse:0.14931                                                         
[104]	test-rmse:0.14934     

[15]	test-rmse:0.15864                                                          
[16]	test-rmse:0.15890                                                          
[17]	test-rmse:0.15912                                                          
[18]	test-rmse:0.15948                                                          
[19]	test-rmse:0.15980                                                          
[20]	test-rmse:0.15995                                                          
[21]	test-rmse:0.16016                                                          
[22]	test-rmse:0.16039                                                          
[23]	test-rmse:0.16056                                                          
[24]	test-rmse:0.16072                                                          
[25]	test-rmse:0.16097                                                          
[26]	test-rmse:0.16114                                                          
[27]	test-rmse:0.16120      

[116]	test-rmse:0.16437                                                         
[117]	test-rmse:0.16437                                                         
[118]	test-rmse:0.16437                                                         
[119]	test-rmse:0.16437                                                         
[120]	test-rmse:0.16437                                                         
[121]	test-rmse:0.16437                                                         
[122]	test-rmse:0.16437                                                         
[123]	test-rmse:0.16437                                                         
[124]	test-rmse:0.16437                                                         
[125]	test-rmse:0.16437                                                         
[126]	test-rmse:0.16437                                                         
[127]	test-rmse:0.16437                                                         
[128]	test-rmse:0.16437     

## Pickle file

In [139]:
import pickle

with open ('model.bin', 'wb') as file:
    pickle.dump(rfr, file)